## 准备数据

In [24]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [25]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))
x, y = mnist_dataset()
print(x[0].shape)
print(x[1].shape)

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]
(60000, 28, 28)
(60000,)


## 建立模型

In [26]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        w, h = 28, 28
        hidden_dim = 256
        y_dim = 10
        self.faltten = layers.Flatten(input_shape=(w, h))
        self.W1 = tf.Variable(tf.random.normal([w*h, hidden_dim], stddev=0.01), trainable=True)
        self.b1 = tf.Variable(tf.zeros([hidden_dim]), trainable=True)
        self.W2 = tf.Variable(tf.random.normal([hidden_dim, y_dim], stddev=0.01), trainable=True)
        self.b2 = tf.Variable(tf.zeros([y_dim]), trainable=True)
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x_flat = self.faltten(x)
        h1 = tf.matmul(x_flat, self.W1) + self.b1
        a1 = tf.tanh(h1)
        logits = tf.matmul(a1, self.W2) + self.b2        
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

/root/autodl-tmp/nndl_exercise/.conda/lib/python3.9/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


## 计算 loss

In [27]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [28]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.3056855 ; accuracy 0.052816667
epoch 1 : loss 2.305117 ; accuracy 0.055983335
epoch 2 : loss 2.304549 ; accuracy 0.058683332
epoch 3 : loss 2.3039813 ; accuracy 0.062016666
epoch 4 : loss 2.3034143 ; accuracy 0.06525
epoch 5 : loss 2.3028479 ; accuracy 0.06853333
epoch 6 : loss 2.3022811 ; accuracy 0.07216667
epoch 7 : loss 2.3017154 ; accuracy 0.07635
epoch 8 : loss 2.3011498 ; accuracy 0.0806
epoch 9 : loss 2.3005843 ; accuracy 0.084983334
epoch 10 : loss 2.3000193 ; accuracy 0.09018333
epoch 11 : loss 2.2994542 ; accuracy 0.09661666
epoch 12 : loss 2.2988892 ; accuracy 0.1036
epoch 13 : loss 2.298324 ; accuracy 0.11065
epoch 14 : loss 2.297759 ; accuracy 0.119333334
epoch 15 : loss 2.2971942 ; accuracy 0.12793334
epoch 16 : loss 2.2966292 ; accuracy 0.13726667
epoch 17 : loss 2.2960644 ; accuracy 0.1473
epoch 18 : loss 2.2954988 ; accuracy 0.15891667
epoch 19 : loss 2.2949333 ; accuracy 0.17045
epoch 20 : loss 2.2943678 ; accuracy 0.18281667
epoch 21 : loss 2.293801